In [1]:
import tensorflow as tf
import os
import time
from matplotlib import pyplot as plt
from IPython import display
import numpy as np
# Utilities for working with remote data
import requests
from io import BytesIO
import zipfile
# Image processing shortcuts
import imageio
import cv2
import copy
from imageio import get_writer
import tensorflow as tf
from IPython import display
from PIL import Image
from google.colab import files
from google.colab.patches import cv2_imshow
from keras.preprocessing.image import save_img

In [3]:
#define loading of input images function:
#load split the input image composed of: input image-real image and mask
def load(image_file):
  w = tf.shape(image_file)[1]
  w = w // 3
  real_image = image_file[:, :w, :]
  input_image = image_file[:,w:2*w , :]
  mask=image_file[:,2*w:,:]
  return input_image, real_image,mask
# normalizing the images to [0, 1]
def normalize(input_image, real_image,mask):
  input_image=input_image/255
  real_image = real_image /255
  mask = mask / 255
  return input_image, real_image,mask
@tf.function()
def load_image_train(image_file):
  input_image, real_image,mask = load(image_file)
  input_image, real_image,mask = normalize(input_image, real_image,mask)
  return input_image, real_image,mask

In [7]:
#import the model, if using colab first upload it
from keras import models    
model = models.Sequential()
modelproj=tf.keras.models.load_model(
    'modelproj.h5', custom_objects=None, compile=True, options=None
)

In [4]:
#uploading of the volume that is going to be tested and creation of a testing dataset
ctrp = requests.get(
    'http://144.91.118.156/datoreteproj_1.zip')
ctzip = BytesIO(ctrp.content)
ct_fdata = zipfile.ZipFile(ctzip)
proj = np.array(
    [imageio.imread(ct_fdata.open(fname)) for fname in ct_fdata.namelist()])
prova_data=[]
proj =tf.expand_dims(proj, axis=3)
for j in range(360):
  prova_data.append(proj[j,:,:,:])
prova_dataset = tf.data.Dataset.from_tensor_slices(prova_data)
prova_dataset = prova_dataset.map(load_image_train,
                                  num_parallel_calls=tf.data.AUTOTUNE)
prova_dataset = prova_dataset.batch(1)

In [ ]:
#delete images if the code have been runned before
#for i in range(360):
#  j=i+100
#  %rm 'projcorr_{j}.png'

In [6]:
#defining a loss function between output image and target image
def se(imageA, imageB):
	# the 'Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
  A=np.array(imageA)
  B=np.array(imageB)
  err = np.sum((A - B) ** 2)
  return err

In [ ]:
#downloading the output of the model (in a zip file) for the testing dataset and saving the SE error 
j=100
k=0
error=np.zeros(360)
import PIL
for inp, tar,mask in prova_dataset:
   G_x =modelproj([inp,mask], training=False)
   ycap=tf.math.multiply(mask,G_x)+tf.math.multiply((tf.ones_like(mask)- mask),inp)
   S=se(tar,ycap)
   error[k]=S
   k=k+1
   yc=np.array(ycap)
   #Gx=np.array(G_x) #if the generetor output wnats to be saved instead of ycap
   yc=yc*255
   #G=Gx*255
   tensc = yc.astype(np.uint8)
   tensc = tensc[0,:,:,0]
   imgc = PIL.Image.fromarray(tensc)
   imgc.save(f'projcorr_{j}.png' )
  
   j=j+1
!zip -r projcorr27_1.zip . -i 'projcorr_*.png'
files.download( 'projcorr27_1.zip' )




In [ ]:
#print some value for error
print(error)
print(np.max(error))
print(np.mean(error))
print(np.min(error))

In [10]:
#only saving the SE error without downloading the output
j=0
error7=np.zeros(360)
import PIL
for inp, tar,mask in prova_dataset:
  G_x = modelproj([inp,mask], training=False)
  ycap=tf.math.multiply(mask,G_x)+tf.math.multiply((tf.ones_like(mask)- mask),inp)
  S=se(tar
  ,ycap) 
  error7[j]=S
  j=j+1

In [ ]:
#showing the generator output for one image (or the output)
for inp, tar,mask in prova_dataset.take(1):
  G_x=modelproj([inp,mask], training=False)
  #ycap=tf.math.multiply(mask,G_x)+tf.math.multiply((tf.ones_like(mask)- mask),inp)
  plt.figure(figsize=(8, 8), dpi=100)
  plt.imshow(G_x[0,:,:,0], cmap=plt.cm.gray)

In [14]:
#defining a function that create a mask of the border of the mask
def image_bord(mask):
  image_border = np.zeros((384,384))   
  for i in range(384):
    for j in range(384): 
      if mask[i,j]==1:
       image_border[i-4:i+4,j]=1
       image_border[i,j-4:j+4]=1
  for i in range(384):
    for j in range(384): 
      if mask[i,j]==1:
        image_border[i,j]=0
  return image_border


In [19]:
# donwloading and saving SE error of the output after interpolating the border (linear interpolation)
l=100 
k=0
error2=np.zeros(360)
import scipy
import PIL
for inp, tar,mask in prova_dataset.take(1):
   G_x =modelproj([inp,mask], training=False)
   ycap=tf.math.multiply(mask,G_x)+tf.math.multiply((tf.ones_like(mask)- mask),inp)
   S=se(tar,ycap)
   error2[k]=S
   k=k+1
   out=ycap.numpy()
   ma=mask.numpy()
   ma=ma[0,:,:,0]
   ta=tar.numpy()
   ta=ta[0,:,:,0]
   out=out[0,:,:,0]
   bord=image_bord(ma)
   for i in range(384):
    for j in range(384): 
      if bord[i,j]==1:
        out[i,j]=(out[i+1,j]+out[i-1,j]+out[i,j+1]+out[i,j-1])/4
  # G=G*255
#showing the corrected image for one image
#plt.figure(figsize=(8, 8), dpi=100)
#plt.imshow(bord, cmap=plt.cm.gray)
#plt.figure(figsize=(8, 8), dpi=100)
#plt.imshow(out, cmap=plt.cm.gray)

#downloading all the correccted image
   tensc = G.astype(np.uint8)

   imgc = PIL.Image.fromarray(tensc)
   imgc.save(f'projcorr_{l}.png' )
   l=l+1
!zip -r projcorr_4.zip . -i 'ycap_*.png'
files.download( 'projcorr_4.zip' )



	zip warning: zip file empty


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#make boxplot figures of the SE error over different testing dataset or model

fig, ax = plt.pyplot.subplots(figsize=(12, 7))
# Remove top and right border
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
# Remove y-axis tick marks
ax.yaxis.set_ticks_position('none')
ax.set_ylabel('SE error')
# Set plot title
ax.set_title('Distribution of SE error in a set of projections')
# Add major gridlines in the y-axis
ax.grid(color='grey', axis='y', linestyle='-', linewidth=0.25, alpha=0.5)


# Set the colors for each distribution
colors = ['#73020C', '#426A8C', '#D94D1A']
colors_setosa = dict(color=colors[0])
colors_versicolor = dict(color=colors[1])
colors_virginica = dict(color=colors[2])

ax.boxplot(error1, positions=[1] ,labels=[('basis ')], boxprops=colors_setosa, medianprops=colors_setosa, whiskerprops=colors_setosa, capprops=colors_setosa, flierprops=dict(markeredgecolor=colors[0]))
ax.boxplot(error5, positions=[2], labels=[('basis ')], boxprops=colors_virginica, medianprops=colors_virginica, whiskerprops=colors_virginica, capprops=colors_virginica, flierprops=dict(markeredgecolor=colors[2]))
ax.boxplot(error3, positions=[3],labels=[('L2norm ')], boxprops=colors_setosa, medianprops=colors_setosa, whiskerprops=colors_setosa, capprops=colors_setosa, flierprops=dict(markeredgecolor=colors[0]))
ax.boxplot(error4, positions=[4] ,labels=[('L2norm ')],  boxprops=colors_virginica, medianprops=colors_virginica, whiskerprops=colors_virginica, capprops=colors_virginica, flierprops=dict(markeredgecolor=colors[2]))
ax.boxplot(error7, positions=[5], labels=[('L2norm_LR')], boxprops=colors_setosa, medianprops=colors_setosa, whiskerprops=colors_setosa, capprops=colors_setosa, flierprops=dict(markeredgecolor=colors[0]))
ax.boxplot(error6, positions=[6],labels=[('L2norm_LR')], boxprops=colors_virginica, medianprops=colors_virginica, whiskerprops=colors_virginica, capprops=colors_virginica, flierprops=dict(markeredgecolor=colors[2]))
plt.pyplot.show()